In [0]:
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os, csv
import sys
import json
import logging
import shutil

import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils import data
import torchvision 
from torchvision import transforms
from torch.optim.lr_scheduler import ExponentialLR
import tqdm

!pip install -U tensorboardcolab
from tensorboardcolab import TensorBoardColab

import scipy.signal
from scipy.io import wavfile
!pip install librosa
import librosa

!pip install https://github.com/vBaiCai/python-pesq/archive/master.zip
!pip install pystoi
!pip install soundfile
import soundfile as sf
from pystoi.stoi import stoi
from pypesq import pesq

!pip install GoogleDriveDownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

Requirement already up-to-date: tensorboardcolab in /usr/local/lib/python3.6/dist-packages (0.0.22)
     / 204kB 775kB/s
  Created wheel for pypesq: filename=pypesq-1.0-cp36-cp36m-linux_x86_64.whl size=80150 sha256=505ca63e7bcd6aae0a845d41a6d831331e16befe7c340458b61f21a11806515e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6djqhn0e/wheels/7b/1c/45/917e2bebf14f475e220194afeebb731d74215e10698e89a620
Successfully built pypesq


# Training tools

## Data process

In [0]:
# Download Train set
gdd.download_file_from_google_drive(file_id='1CULVCAq0T3wqZTPGIqPja6OtwjYJkGAy',
                                    dest_path='dataset/train.tar.gz',
                                    unzip=False, showsize=True)
get_ipython().system('tar xvzf dataset/train.tar.gz')
# In[53]:


# Download Valid set
gdd.download_file_from_google_drive(file_id='1WE229Jt9WV2iZbxY7YjkYIfSZyCHz9Iq',
                                    dest_path='dataset/valid.tar.gz',
                                    unzip=False, showsize=True)
get_ipython().system('tar xvzf dataset/valid.tar.gz')
# In[ ]:

### Dataloader

In [0]:
def load_data_list(folder='/home/sangwon/바탕화면/dataset', setname='train'):
    assert(setname in ['train', 'valid'])

    dataset = {}
    # foldername = folder + '/' + setname + 'set'
    foldername = folder + '/' + setname
    # print(foldername) # /home/sangwon/바탕화면/dataset/valid
    

    print("Loading files...")
    dataset['innames'] = []
    dataset['outnames'] = []
    dataset['shortnames'] = []

    noisy_filelist = os.listdir("%s_noisy"%(foldername))
    # filelist = [f for f in filelist if f.endswith(".wav")]
    for i in tqdm(noisy_filelist):
        dataset['innames'].append("%s_noisy/%s"%(foldername,i))
        dataset['shortnames'].append("%s"%(i))
        
    clean_filelist = os.listdir("%s_clean"%(foldername))
    for i in tqdm(clean_filelist):
        dataset['outnames'].append("%s_clean/%s"%(foldername,i))

    return dataset

# DATA LOADING - LOAD FILE DATA
def load_data(dataset):

    dataset['inaudio']  = [None]*len(dataset['innames'])
    dataset['outaudio'] = [None]*len(dataset['outnames'])

    for id in tqdm(range(len(dataset['innames']))):

        if dataset['inaudio'][id] is None:
            inputData, sr = librosa.load(dataset['innames'][id], sr=None)
            outputData, sr = librosa.load(dataset['outnames'][id], sr=None)

            shape = np.shape(inputData)

            dataset['inaudio'][id]  = np.float32(inputData)
            dataset['outaudio'][id] = np.float32(outputData)

    return dataset

class AudioDataset(data.Dataset):
    """
    Audio sample reader.
    """

    def __init__(self, data_type):
        dataset = load_data_list(setname=data_type)
        self.dataset = load_data(dataset)

        self.file_names = dataset['innames']

    def __getitem__(self, idx):
        mixed = torch.from_numpy(self.dataset['inaudio'][idx]).type(torch.FloatTensor)
        clean = torch.from_numpy(self.dataset['outaudio'][idx]).type(torch.FloatTensor)

        return mixed, clean

    def __len__(self):
        return len(self.file_names)

    def zero_pad_concat(self, inputs):
        max_t = max(inp.shape[0] for inp in inputs)
        shape = (len(inputs), max_t)
        input_mat = np.zeros(shape, dtype=np.float32)
        for e, inp in enumerate(inputs):
            input_mat[e, :inp.shape[0]] = inp
        return input_mat

    def collate(self, inputs):
        mixeds, cleans = zip(*inputs)
        seq_lens = torch.IntTensor([i.shape[0] for i in mixeds])

        x = torch.FloatTensor(self.zero_pad_concat(mixeds))
        y = torch.FloatTensor(self.zero_pad_concat(cleans))

        batch = [x, y, seq_lens]
        return batch


# In[4]:


# train_dataset = AudioDataset(data_type='train')
# train_data_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
#         collate_fn=train_dataset.collate, shuffle=True, num_workers=4)
valid_dataset = AudioDataset(data_type='valid')


# In[5]:


valid_data_loader = DataLoader(dataset=test_dataset, batch_size=4,
        collate_fn=test_dataset.collate, shuffle=False, num_workers=4)


# In[6]:


for tdl in iter(valid_data_loader):
    print(tdl)
    print(len(tdl))
    print(tdl[0].size())
    print(tdl[1].size())
    print(tdl[2].size())
    break


# In[ ]:

Loading files...


FileNotFoundError: ignored

## Script

### complexnn

In [0]:
# Utility functions for initialization
def complex_rayleigh_init(Wr, Wi, fanin=None, gain=1):
    if not fanin:
        fanin = 1
        for p in W1.shape[1:]: fanin *= p
    scale  = float(gain) / float(fanin)
    theta  = torch.empty_like(Wr).uniform_(-math.pi/2, +math.pi/2)
    rho    = np.random.rayleigh(scale, tuple(Wr.shape))
    rho    = torch.tensor(rho).to(Wr)
    Wr.data.copy_(rho * theta.cos())
    Wi.data.copy_(rho * theta.sin())

# Layers
class ComplexConvWrapper(nn.Module):
    def __init__(self, conv_module, *args, **kwargs):
        super(ComplexConvWrapper, self).__init__()
        self.conv_re = conv_module(*args, **kwargs)
        self.conv_im = conv_module(*args, **kwargs)

    def reset_parameters(self):
        fanin = self.conv_re.in_channels // self.conv_re.groups
        for s in self.conv_re.kernel_size: fanin *= s
        complex_rayleigh_init(self.conv_re.weight, self.conv_im.weight, fanin)
        if self.conv_re.bias is not None:
            self.conv_re.bias.data.zero_()
            self.conv_im.bias.data.zero_()

    def forward(self, xr, xi):
        real = self.conv_re(xr) - self.conv_im(xi)
        imag = self.conv_re(xi) + self.conv_im(xr)
        return real, imag

# Real-valued network module for complex input
class RealConvWrapper(nn.Module):
    def __init__(self, conv_module, *args, **kwargs):
        super(ComplexConvWrapper,self).__init__()
        self.conv_re = conv_module(*args, **kwargs)

    def forward(self, xr, xi):
        real = self.conv_re(xr)
        imag = self.conv_re(xi)
        return real, imag

class CLeakyReLU(nn.LeakyReLU):
    def forward(self, xr, xi):
        return F.leaky_relu(xr, self.negative_slope, self.inplace),\
                F.leaky_relu(xi, self.negative_slope, self.inplace)

# Source: https://github.com/ChihebTrabelsi/deep_complex_networks/tree/pytorch
class ComplexBatchNorm(torch.nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True,
            track_running_stats=True):
        super(ComplexBatchNorm, self).__init__()
        self.num_features        = num_features
        self.eps                 = eps
        self.momentum            = momentum
        self.affine              = affine
        self.track_running_stats = track_running_stats
        if self.affine:
            self.Wrr = torch.nn.Parameter(torch.Tensor(num_features))
            self.Wri = torch.nn.Parameter(torch.Tensor(num_features))
            self.Wii = torch.nn.Parameter(torch.Tensor(num_features))
            self.Br  = torch.nn.Parameter(torch.Tensor(num_features))
            self.Bi  = torch.nn.Parameter(torch.Tensor(num_features))
        else:
            self.register_parameter('Wrr', None)
            self.register_parameter('Wri', None)
            self.register_parameter('Wii', None)
            self.register_parameter('Br',  None)
            self.register_parameter('Bi',  None)
        if self.track_running_stats:
            self.register_buffer('RMr',  torch.zeros(num_features))
            self.register_buffer('RMi',  torch.zeros(num_features))
            self.register_buffer('RVrr', torch.ones (num_features))
            self.register_buffer('RVri', torch.zeros(num_features))
            self.register_buffer('RVii', torch.ones (num_features))
            self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))
        else:
            self.register_parameter('RMr',                 None)
            self.register_parameter('RMi',                 None)
            self.register_parameter('RVrr',                None)
            self.register_parameter('RVri',                None)
            self.register_parameter('RVii',                None)
            self.register_parameter('num_batches_tracked', None)
        self.reset_parameters()

    def reset_running_stats(self):
        if self.track_running_stats:
            self.RMr.zero_()
            self.RMi.zero_()
            self.RVrr.fill_(1)
            self.RVri.zero_()
            self.RVii.fill_(1)
            self.num_batches_tracked.zero_()

    def reset_parameters(self):
        self.reset_running_stats()
        if self.affine:
            self.Br.data.zero_()
            self.Bi.data.zero_()
            self.Wrr.data.fill_(1)
            self.Wri.data.uniform_(-.9, +.9) # W will be positive-definite
            self.Wii.data.fill_(1)

    def _check_input_dim(self, xr, xi):
        assert(xr.shape == xi.shape)
        assert(xr.size(1) == self.num_features)

    def forward(self, xr, xi):
        self._check_input_dim(xr, xi)

        exponential_average_factor = 0.0

        if self.training and self.track_running_stats:
            self.num_batches_tracked += 1
            if self.momentum is None:  # use cumulative moving average
                exponential_average_factor = 1.0 / self.num_batches_tracked.item()
            else:  # use exponential moving average
                exponential_average_factor = self.momentum

        #
        # NOTE: The precise meaning of the "training flag" is:
        #       True:  Normalize using batch   statistics, update running statistics
        #              if they are being collected.
        #       False: Normalize using running statistics, ignore batch   statistics.
        #
        training = self.training or not self.track_running_stats
        redux = [i for i in reversed(range(xr.dim())) if i!=1]
        vdim  = [1] * xr.dim()
        vdim[1] = xr.size(1)

        #
        # Mean M Computation and Centering
        #
        # Includes running mean update if training and running.
        #
        if training:
            Mr, Mi = xr, xi
            for d in redux:
                Mr = Mr.mean(d, keepdim=True)
                Mi = Mi.mean(d, keepdim=True)
            if self.track_running_stats:
                self.RMr.lerp_(Mr.squeeze(), exponential_average_factor)
                self.RMi.lerp_(Mi.squeeze(), exponential_average_factor)
        else:
            Mr = self.RMr.view(vdim)
            Mi = self.RMi.view(vdim)
        xr, xi = xr-Mr, xi-Mi

        #
        # Variance Matrix V Computation
        #
        # Includes epsilon numerical stabilizer/Tikhonov regularizer.
        # Includes running variance update if training and running.
        #
        if training:
            Vrr = xr * xr
            Vri = xr * xi
            Vii = xi * xi
            for d in redux:
                Vrr = Vrr.mean(d, keepdim=True)
                Vri = Vri.mean(d, keepdim=True)
                Vii = Vii.mean(d, keepdim=True)
            if self.track_running_stats:
                self.RVrr.lerp_(Vrr.squeeze(), exponential_average_factor)
                self.RVri.lerp_(Vri.squeeze(), exponential_average_factor)
                self.RVii.lerp_(Vii.squeeze(), exponential_average_factor)
        else:
            Vrr = self.RVrr.view(vdim)
            Vri = self.RVri.view(vdim)
            Vii = self.RVii.view(vdim)
        Vrr   = Vrr + self.eps
        Vri   = Vri
        Vii   = Vii + self.eps

        #
        # Matrix Inverse Square Root U = V^-0.5
        #
        # sqrt of a 2x2 matrix,
        # - https://en.wikipedia.org/wiki/Square_root_of_a_2_by_2_matrix
        tau   = Vrr + Vii
        delta = torch.addcmul(Vrr * Vii, -1, Vri, Vri)
        s     = delta.sqrt()
        t     = (tau + 2*s).sqrt()

        # matrix inverse, http://mathworld.wolfram.com/MatrixInverse.html
        rst   = (s * t).reciprocal()
        Urr   = (s + Vii) * rst
        Uii   = (s + Vrr) * rst
        Uri   = (  - Vri) * rst

        #
        # Optionally left-multiply U by affine weights W to produce combined
        # weights Z, left-multiply the inputs by Z, then optionally bias them.
        #
        # y = Zx + B
        # y = WUx + B
        # y = [Wrr Wri][Urr Uri] [xr] + [Br]
        #     [Wir Wii][Uir Uii] [xi]   [Bi]
        #
        if self.affine:
            Wrr, Wri, Wii = self.Wrr.view(vdim), self.Wri.view(vdim), self.Wii.view(vdim)
            Zrr = (Wrr * Urr) + (Wri * Uri)
            Zri = (Wrr * Uri) + (Wri * Uii)
            Zir = (Wri * Urr) + (Wii * Uri)
            Zii = (Wri * Uri) + (Wii * Uii)
        else:
            Zrr, Zri, Zir, Zii = Urr, Uri, Uri, Uii

        yr = (Zrr * xr) + (Zri * xi)
        yi = (Zir * xr) + (Zii * xi)

        if self.affine:
            yr = yr + self.Br.view(vdim)
            yi = yi + self.Bi.view(vdim)

        return yr, yi

    def extra_repr(self):
        return '{num_features}, eps={eps}, momentum={momentum}, affine={affine}, ' \
                'track_running_stats={track_running_stats}'.format(**self.__dict__)

### Unet

In [0]:
# NOTE: Use Complex Ops for DCUnet when implemented
# Reference:
#  > Progress: https://github.com/pytorch/pytorch/issues/755
def pad2d_as(x1, x2):
    # Pad x1 to have same size with x2
    # inputs are NCHW
    diffH = x2.size()[2] - x1.size()[2]
    diffW = x2.size()[3] - x1.size()[3]

    return F.pad(x1, (0, diffW, 0, diffH)) # (L,R,T,B)

def padded_cat(x1, x2, dim):
    # NOTE: Use torch.cat with pad instead when merged
    #  > https://github.com/pytorch/pytorch/pull/11494
    x1 = pad2d_as(x1, x2)
    x1 = torch.cat([x1, x2], dim=dim)
    return x1

class Encoder(nn.Module):
    def __init__(self, conv_cfg, leaky_slope):
        super(Encoder, self).__init__()
        self.conv = ComplexConvWrapper(nn.Conv2d, *conv_cfg, bias=False)
        self.bn = ComplexBatchNorm(conv_cfg[1])
        self.act = CLeakyReLU(leaky_slope, inplace=True)

    def forward(self, xr, xi):
        xr, xi = self.act(*self.bn(*self.conv(xr, xi)))
        return xr, xi

class Decoder(nn.Module):
    def __init__(self, dconv_cfg, leaky_slope):
        super(Decoder, self).__init__()
        self.dconv = ComplexConvWrapper(nn.ConvTranspose2d, *dconv_cfg, bias=False)
        self.bn = ComplexBatchNorm(dconv_cfg[1])
        self.act = CLeakyReLU(leaky_slope, inplace=True)

    def forward(self, xr, xi, skip=None):
        if skip is not None:
            xr, xi = padded_cat(xr, skip[0], dim=1), padded_cat(xi, skip[1], dim=1)
        xr, xi = self.act(*self.bn(*self.dconv(xr, xi)))
        return xr, xi

class Unet(nn.Module):
    def __init__(self, cfg):
        super(Unet, self).__init__()
        self.encoders = nn.ModuleList()
        for conv_cfg in cfg['encoders']:
            self.encoders.append(Encoder(conv_cfg, cfg['leaky_slope']))

        self.decoders = nn.ModuleList()
        for dconv_cfg in cfg['decoders'][:-1]:
            self.decoders.append(Decoder(dconv_cfg, cfg['leaky_slope']))

        # Last decoder doesn't use BN & LeakyReLU. Use bias.
        self.last_decoder = ComplexConvWrapper(nn.ConvTranspose2d,
                *cfg['decoders'][-1], bias=True)

        self.ratio_mask_type = cfg['ratio_mask']

    def get_ratio_mask(self, outr, outi):
        def inner_fn(r, i):
            if self.ratio_mask_type == 'BDSS':
                return torch.sigmoid(outr) * r, torch.sigmoid(outi) * i
            else:
                # Polar cordinate masks
                # x1.4 slower
                mag_mask = torch.sqrt(outr**2 + outi**2)
                # M_phase = O/|O| for O = g(X)
                # Same phase rotate(theta), for phase mask O/|O| and O.
                phase_rotate = torch.atan2(outi, outr)

                if self.ratio_mask_type == 'BDT':
                    mag_mask = torch.tanh(mag_mask)
                # else then UBD(Unbounded)

                mag = mag_mask * torch.sqrt(r**2 + i**2)
                phase = phase_rotate + torch.atan2(i, r)

                # return real, imag
                return mag * torch.cos(phase), mag * torch.sin(phase)

        return inner_fn

    def forward(self, xr, xi):
        input_real, input_imag = xr, xi
        skips = list()

        for encoder in self.encoders:
            xr, xi = encoder(xr, xi)
            skips.append((xr, xi))

        skip = skips.pop()
        skip = None # First decoder input x is same as skip, drop skip.
        for decoder in self.decoders:
            xr, xi = decoder(xr, xi, skip)
            skip = skips.pop()

        xr, xi = padded_cat(xr, skip[0], dim=1), padded_cat(xi, skip[1], dim=1)
        xr, xi = self.last_decoder(xr, xi)

        xr, xi = pad2d_as(xr, input_real), pad2d_as(xi, input_imag)
        ratio_mask_fn = self.get_ratio_mask(xr, xi)
        return ratio_mask_fn(input_real, input_imag)

### ISTFT

In [0]:
class ISTFT(torch.nn.Module):
    def __init__(self, filter_length=1024, hop_length=512, window='hanning', center=True):
        super(ISTFT, self).__init__()

        self.filter_length = filter_length
        self.hop_length = hop_length
        self.center = center

        win_cof = scipy.signal.get_window(window, filter_length)
        self.inv_win = self.inverse_stft_window(win_cof, hop_length)

        fourier_basis = np.fft.fft(np.eye(self.filter_length))
        cutoff = int((self.filter_length / 2 + 1))
        fourier_basis = np.vstack([np.real(fourier_basis[:cutoff, :]),
                                   np.imag(fourier_basis[:cutoff, :])])
        inverse_basis = torch.FloatTensor(self.inv_win * \
                np.linalg.pinv(fourier_basis).T[:, None, :])

        self.register_buffer('inverse_basis', inverse_basis.float())

    # Use equation 8 from Griffin, Lim.
    # Paper: "Signal Estimation from Modified Short-Time Fourier Transform"
    # Reference implementation: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/signal/spectral_ops.py
    # librosa use equation 6 from paper: https://github.com/librosa/librosa/blob/0dcd53f462db124ed3f54edf2334f28738d2ecc6/librosa/core/spectrum.py#L302-L311
    def inverse_stft_window(self, window, hop_length):
        window_length = len(window)
        denom = window ** 2
        overlaps = -(-window_length // hop_length)  # Ceiling division.
        denom = np.pad(denom, (0, overlaps * hop_length - window_length), 'constant')
        denom = np.reshape(denom, (overlaps, hop_length)).sum(0)
        denom = np.tile(denom, (overlaps, 1)).reshape(overlaps * hop_length)
        return window / denom[:window_length]

    def forward(self, real_part, imag_part, length=None):
        if (real_part.dim() == 2):
            real_part = real_part.unsqueeze(0)
            imag_part = imag_part.unsqueeze(0)

        recombined = torch.cat([real_part, imag_part], dim=1)

        inverse_transform = F.conv_transpose1d(recombined,
                                               self.inverse_basis,
                                               stride=self.hop_length,
                                               padding=0)

        padded = int(self.filter_length // 2)
        if length is None:
            if self.center:
                inverse_transform = inverse_transform[:, :, padded:-padded]
        else:
            if self.center:
                inverse_transform = inverse_transform[:, :, padded:]
            inverse_transform = inverse_transform[:, :, :length]

        return inverse_transform

### Utils

In [0]:
class Params():
    """Class that loads hyperparameters from a json file.
    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']"""
        return self.__dict__


class RunningAverage():
    """A simple class that maintains the running average of a quantity
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)


def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.
    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.
    Example:
    ```
    logging.info("Starting training...")
    ```
    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file
    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


def save_checkpoint(state, is_best, checkpoint):
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'
    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth.tar')
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))


def load_checkpoint(checkpoint, model, optimizer=None):
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.
    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    if not os.path.exists(checkpoint):
        raise ("File doesn't exist {}".format(checkpoint))
    checkpoint = torch.load(checkpoint) 
    model.load_state_dict(checkpoint['state_dict'])
    best_pesq = checkpoint['best_pesq']
    best_stoi = checkpoint['best_stoi']

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return best_pesq, best_stoi

# Training

In [0]:
n_fft, hop_length = 400, 160
window = torch.hann_window(n_fft).cuda()
# STFT
stft = lambda x: torch.stft(x, n_fft, hop_length, window=window)
# ISTFT
istft = ISTFT(n_fft, hop_length, window='hanning').cuda()

In [0]:
# hyper-parameters
max_epoch = 60
learning_rate = 0.001
batch_size = 30
device = 'cuda'

base_param = {"seed" : 2017, "save_path" : "/deep/group/awni/speech_models/test",
    "data" : {
        "train_set" : "/deep/group/speech/datasets/LibriSpeech/train-toy.json",
        "dev_set" : "/deep/group/speech/datasets/LibriSpeech/dev-toy.json"
    },

    "optimizer" : {
        "batch_size" : 8,
        "epochs" : 1000,
        "learning_rate" : 1e-3,
        "momentum" : 0.0
    },

    "model" : {
        "leaky_slope" : 0.1,
        "ratio_mask" : "BDT",
        "encoders" : [
            [1, 32, [7, 5], [2, 2], [3, 2]],
            [32, 32, [7, 5], [2, 1], [3, 2]],
            [32, 64, [7, 5], [2, 2], [3, 2]],
            [64, 64, [5, 3], [2, 1], [2, 1]],
            [64, 64, [5, 3], [2, 2], [2, 1]],
            [64, 64, [5, 3], [2, 1], [2, 1]],
            [64, 64, [5, 3], [2, 2], [2, 1]],
            [64, 64, [5, 3], [2, 1], [2, 1]]
        ],
        "decoders" : [
            [64, 64, [5, 3], [2, 1], [2, 1]],
            [128, 64, [5, 3], [2, 2], [2, 1]],
            [128, 64, [5, 3], [2, 1], [2, 1]],
            [128, 64, [5, 3], [2, 2], [2, 1]],
            [128, 64, [5, 3], [2, 1], [2, 1]],
            [128, 32, [7, 5], [2, 2], [3, 2]],
            [64, 32, [7, 5], [2, 1], [3, 2]],
            [64, 1, [7, 5], [2, 2], [3, 2]]
        ],
        "__coder_keys" : [
            "in_channels", "out_channels", "kernel_size", "stride", "padding"
        ]
    }
}


# initialize tensorboard for visualization
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://a4a882f9.ngrok.io


In [0]:
#data loader
train_dataset = AudioDataset(data_type='train')
test_dataset = AudioDataset(data_type='val')
train_data_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, collate_fn=train_dataset.collate, shuffle=True, num_workers=4)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, collate_fn=test_dataset.collate, shuffle=False, num_workers=4)

In [0]:
#model set
net = Unet(base_param['model']).cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [0]:
#Check point load

ckpt_dir = os.path.join(gdrive_root, 'checkpoints')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

best_pesq = 0.
best_stoi = 0.
ckpt_path = os.path.join(ckpt_dir, 'SEckpt.pt')
if os.path.exists(ckpt_path):
  ckpt = torch.load(ckpt_path)
  try:
    net.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    best_pesq = ckpt['best_pesq']
  except RuntimeError as e:
      print('wrong checkpoint')
  else:    
    print('checkpoint is loaded !')
    print('current best pesq : %.4f' % best_pesq)
    print('current best stoi : %.4f' % best_stoi)

In [0]:
#Train & test
iteration = 0

for epoch in range(max_epoch):
      train_bar = tqdm(train_data_loader)
        for input in train_bar:
            iteration += 1
            #load data
            train_mixed, train_clean, seq_len = map(lambda x: x.cuda(), input)
            mixed = stft(train_mixed).unsqueeze(dim=1)
            real, imag = mixed[..., 0], mixed[..., 1]

            #feed data
            out_real, out_imag = net(real, imag)
            out_real, out_imag = torch.squeeze(out_real, 1), torch.squeeze(out_imag, 1)
            out_audio = istft(out_real, out_imag, train_mixed.size(1))
            out_audio = torch.squeeze(out_audio, dim=1)
            for i, l in enumerate(seq_len):
                out_audio[i, l:] = 0
            librosa.output.write_wav('mixed.wav', train_mixed[0].cpu().data.numpy()[:seq_len[0].cpu().data.numpy()], 16000)
            librosa.output.write_wav('clean.wav', train_clean[0].cpu().data.numpy()[:seq_len[0].cpu().data.numpy()], 16000)
            librosa.output.write_wav('out.wav', out_audio[0].cpu().data.numpy()[:seq_len[0].cpu().data.numpy()], 16000)

            #calculate LOSS
            #loss =  wSDRLoss(train_mixed, train_clean, out_audio)
            loss = torch.nn.MSELoss(out_audio, train_clean)
            #gradient optimizer
            optimizer.zero_grad()

            #backpropagate LOSS
            loss.backward()

            #update weight
            optimizer.step()

            #calculate accuracy
            PESQ = pesq('clean.wav', 'out.wav', 16000)
            STOI = stoi('clean.wav', 'out.wav', 16000)

            #flot tensorboard
            if iteration % "num" == 0:
                summary.add_scalar('Train Loss', loss.item(), iteration)
                tbc.save_value('Loss', 'Train Loss', iteration ,loss.item())
                print('[epoch: {}, iteration: {}] train loss : {:.4f} PESQ : {:.4f} STOI : {:.4f}'.format(epoch, iteration, loss, PESQ, STOI))

            #test phase
            n = 0
            test_loss = 0
            test_acc = 0
            test_bar = tqdm(test_train_loader)
            for input in test_bar:
                test_mixed, test_clean, test_seq_len = map(lambda x: x.cuda(), input)
                logits_real, logits_imag = net(input)
                logits_real, logits_imag = torch.squeeze(logits_real, 1), torch.squeeze(out_imag, 1)
                logits_audio = istft(logits_real, logits_imag, test_mixed.size(1))
                logits_audio = torch.squeeze(logits_audio, dim=1)
                for i, l in enumerate(test_seq_len):
                logits_audio[i, l:] = 0
                librosa.output.write_wav('test_clean.wav', test_clean[0].cpu().data.numpy()[:seq_len[0].cpu().data.numpy()], 16000)
                librosa.output.write_wav('test_out.wav', logits_audio[0].cpu().data.numpy()[:seq_len[0].cpu().data.numpy()], 16000)

                #test loss
                #test_loss = wSDRLoss(test_mixed, test_clean, logits_audio)
                test_loss += torch.nn.MSELoss(logits_audio, test_clean, reduction = 'sum').item()
                #test accuracy
                test_pesq = pesq('test_clean.wav', 'test_out.wav', 16000)
                test_stoi = stoi('test_clean.wav', 'test_out.wav', 16000)

            tbc.save_value('Loss', 'Test_loss', iteration, test_loss)
            print('[epoch: {}, iteration: {}] test loss : {:.4f} PESQ : {:.4f} STOI : {:.4f}'.format(epoch, iteration, test_loss, test_pesq, test_stoi))
            tbc.flush_line('Train loss')
            tbc.flush_line('Test loss')
            
            if test_pesq > best_pesq or test_stoi > best_stoi:
                best_pesq = test_pesq
                best_stoi = test_stoi
                # Note: optimizer also has states ! don't forget to save them as well.
                ckpt = {'model':net.state_dict(),
                        'optimizer':optimizer.state_dict(),
                        'best_pesq':best_pesq}
                torch.save(ckpt, ckpt_path)
                print('checkpoint is saved !')

tbc.close()